In [62]:
import os
import sys
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
utils_dir = os.path.join(parent_dir, "src", "utils")
sys.path.append(utils_dir)

In [63]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from process_data import process_data
from weighted_accuracy import weighted_accuracy_scorer

In [64]:
X_train_path= os.path.join("..", "data","original_input", "X_train_Wwou3IE.csv")
X_train_preprocessed = pd.read_csv(X_train_path, delimiter=',')
X_test_path= os.path.join("..", "data","original_input", "X_test_GgyECq8.csv")
X_test_preprocessed = pd.read_csv(X_test_path, delimiter=',')
y_train_path= os.path.join("..", "data","original_input", "y_train_jJtXgMX.csv")
y_train_preprocessed = pd.read_csv(y_train_path, delimiter=',')

X_train_preprocessed

,DELIVERY_START,load_forecast,coal_power_available,gas_power_available,nucelear_power_available,wind_power_forecasts_average,solar_power_forecasts_average,wind_power_forecasts_std,solar_power_forecasts_std,predicted_spot_price
0,2022-01-01 02:00:00+01:00,49439.0,3386.0,11487.0,44118.0,3035.0,0.0,79.248348,0.000000,NaN
1,2022-01-01 03:00:00+01:00,46511.0,3386.0,11487.0,44118.0,3143.0,0.0,61.776532,0.000000,NaN
2,2022-01-01 04:00:00+01:00,45158.0,3386.0,11487.0,44118.0,3288.0,0.0,44.291112,0.000000,NaN
3,2022-01-01 05:00:00+01:00,44779.0,3386.0,11487.0,44118.0,3447.0,0.0,36.127588,0.000000,NaN
4,2022-01-01 06:00:00+01:00,45284.0,3386.0,11487.0,44118.0,3679.0,0.0,30.983023,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...
10600,2023-03-29 19:00:00+02:00,50814.0,3386.0,11952.0,38320.0,7552.0,651.0,247.408490,7.821622,108.11
10601,2023-03-29 20:00:00+02:00,50628.0,3386.0,11952.0,38320.0,8338.0,109.0,155.795012,2.534054,125.66
10602,2023-03-29 21:00:00+02:00,48201.0,3386.0,11952.0,38320.0,9115.0,0.0,126.884684,0.000000,138.01
10603,2023-03-29 22:00:00+02:00,47967.0,3386.0,11952.0,38320.0,9636.0,0.0,156.669189,0.000000,136.74


In [65]:
delivery_start = X_test_preprocessed["DELIVERY_START"]

In [66]:
X_train = process_data(X_train_preprocessed.copy(deep=True), "predicted_spot_price", None, "standard")
X_test = process_data(X_test_preprocessed.copy(deep=True), "predicted_spot_price", None, "standard")
y_train = process_data(y_train_preprocessed.copy(deep=True), None, None, None)

In [67]:
from sktime.forecasting.compose import make_reduction
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.base import ForecastingHorizon
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np

regressor = LinearRegression()

# If you want to apply Sequential Feature Selection
sfs = SequentialFeatureSelector(regressor, n_features_to_select="auto", direction="forward")
sfs.fit(X_train, y_train)

# Transform the training and testing sets to only include the selected features
X_train_selected = sfs.transform(X_train)
X_test_selected = sfs.transform(X_test)

# Now, fit the regressor on the selected features from the training data
regressor.fit(X_train_selected, y_train)

# Use the trained model to make predictions on the test set
y_pred = regressor.predict(X_test_selected)

In [68]:
## Assuming 'y' is your time series data
## For demonstration, let's create a dummy time series
#
#X_test.index.freq = 'H'
##y_test.index.freq = 'H'
#
## Define the forecasting horizon
#fh = ForecastingHorizon(X_test.index, is_relative=False)
##fh = ForecastingHorizon(y_test.index, is_relative=False)
#
## Reduction: from forecasting to regression
#regressor = LinearRegression()
#forecaster = make_reduction(regressor, window_length=10, strategy="recursive")
#
## Fit the forecaster
#forecaster.fit(y_train, fh=fh)
#
## Apply Sequential Feature Selector (Note: This is conceptual, as feature selection applies to the regressor in the reduction approach)
## Assuming you have additional exogenous features in 'X' that you want to select features from
## For demonstration, create dummy features)  # Assuming 5 potential features
#
## SFS on the regressor inside the reduction
#sfs = SequentialFeatureSelector(regressor, n_features_to_select="auto", direction="forward")
#sfs.fit(X_train, y_train)
#
## Transform the training and testing sets to only include the selected features
#X_train_selected = sfs.transform(X_train)
#X_test_selected = sfs.transform(X_test)
#
## Normally, you would now fit the regressor on X_train_selected, but with sktime's reduction,
## we need to integrate SFS more carefully, possibly by creating a custom pipeline that combines
## reduction and feature selection for forecasting.

# Predict (here, using the original forecaster without SFS for simplicity)
#y_pred = forecaster.predict(fh)

# Note: The straightforward application of SFS here is conceptual.
# In practice, applying feature selection with sktime's make_reduction
# would require adjusting how features are selected and used, possibly involving custom code
# to handle the temporal nature of forecasting and the exogenous variables.


In [69]:
d = {"DELIVERY_START" : delivery_start, "spot_id_delta" : y_pred.reshape(-1)}
y_pred = pd.DataFrame(data = d)

#y_pred.to_csv("../data/output_y_test/y_test_result_rdf1.csv", index = False)

In [70]:
X_train.isna().sum()

load_forecast                    0
coal_power_available             0
gas_power_available              0
nucelear_power_available         0
wind_power_forecasts_average     0
solar_power_forecasts_average    0
wind_power_forecasts_std         0
solar_power_forecasts_std        0
dtype: int64

In [77]:
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.transformations.series.detrend import Detrender
from sktime.transformations.series.deseasonalize import Deseasonalizer
from sktime.transformations.series.adapt import TabularToSeriesAdaptor
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sktime.forecasting.arima import AutoARIMA
import pandas as pd
import numpy as np

# Example: Manually fit a simplified version of your forecaster
forecaster_simple = TransformedTargetForecaster([
    ("detrender", Detrender()),
    ("deseasonalizer", Deseasonalizer(model="multiplicative", sp=12)),
    ("forecaster", AutoARIMA(suppress_warnings=True, stepwise=True, seasonal=True, sp=12)),
])

# Assuming y_train is your target variable
forecaster_simple.fit(y_train)

# Define the parameter grid for the ARIMA model
param_grid = {
    # Example: tuning the seasonal order of the ARIMA model (P,D,Q,s)
    # "forecaster__seasonal_order": [(1, 1, 1, 12), (0, 1, 1, 12)],
}

# Setup cross-validation
cv = ExpandingWindowSplitter(initial_window=int(len(y_train) * 0.5))

# Setup grid search
gscv = ForecastingGridSearchCV(
    forecaster=forecaster,
    cv=cv,
    param_grid=param_grid,
    n_jobs=-1  # Utilize all available CPUs
)

# Fit the model on the training data
gscv.fit(y_train, fh=fh, X=X_train)

# Print best parameters found by grid search
print("Best Parameters:", gscv.best_params_)

# Predict using the best model for the forecast horizon
y_pred = gscv.predict(fh=fh)

c:\Users\emac\Documents\MachineLearningAvance\elmy\elmy\elmy_env\lib\site-packages\sktime\forecasting\model_selection\_tune.py:90: UserWarning: Parameter n_jobs of ForecastingGridSearchCV has been removed in sktime 0.27.0 and is no longer used. It is ignored when passed. Instead, the backend and backend_params parameters should be used to pass n_jobs or other parallelization parameters.
  warn(


NotFittedError: All fits of forecaster failed,
                set error_score='raise' to see the exceptions.
                Failed forecaster: TransformedTargetForecaster(steps=[('detrender', Detrender()),
                                   ('deseasonalizer',
                                    Deseasonalizer(model='multiplicative',
                                                   sp=12)),
                                   ('scaler',
                                    TabularToSeriesAdaptor(transformer=RobustScaler())),
                                   ('minmax',
                                    TabularToSeriesAdaptor(transformer=MinMaxScaler(feature_range=(1, 10)))),
                                   ('forecaster',
                                    AutoARIMA(sp=12, suppress_warnings=True))])

In [72]:
d = {"DELIVERY_START" : delivery_start, "spot_id_delta" : y_pred.reshape(-1)}
y_pred = pd.DataFrame(data = d)

y_pred.to_csv("../data/output_y_test/y_test_result_LR8.csv", index = False)

AttributeError: 'DataFrame' object has no attribute 'reshape'